In [1]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [2]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

## ניתוח סדרות זמן בניסויים פיזיקליים (Time Series Essentials)

במדידות פיזיקליות, אנו מתמודדים לעיתים קרובות עם **נתונים רציפים בזמן** — כמו זרם, מתח, טמפרטורה או הספק שנמדדים כל כמה שניות, דקות או ימים.  
עיבוד נכון של סדרות זמן מאפשר לנו לחשוף מגמות, תנודות, או חריגות בהתנהגות המערכת לאורך תקופה.

בפנדס קיימים כלים ייעודיים לעבודה עם נתוני זמן: המרה לפורמט תאריך־ושעה, קביעת אינדקס מבוסס זמן, דגימה מחדש (Resampling), חישוב מגמות מחזוריות, והתמודדות עם אזורי זמן שונים.

#### המרה ויצירת DateTimeIndex

נשתמש ב־`pd.to_datetime()` כדי לוודא שעמודת הזמן מיוצגת כ־`datetime64[ns]`.  
לאחר מכן נגדיר אותה כאינדקס באמצעות `set_index("timestamp")` כדי לאפשר ניתוח מבוסס זמן.  
כך נוכל בקלות לחלץ חודשים, שבועות, או ימים, ולבצע חישובי מגמות.

#### דגימה מחדש (Resampling)

פונקציית `resample()` מאפשרת לחשב נתונים מחושבים על פני תקופות זמן — לדוגמה:
- `resample('W').mean()` לחישוב ממוצע שבועי של טמפרטורה או מתח.  
- `resample('M').sum()` לסכום ההספק החודשי הכולל של מערכת.  

הדגימה מחדש חשובה מאוד לצמצום רעשים ולניתוח מגמות רחבות בזמן.

#### ניתוח מגמות מחודש לחודש

באמצעות `pct_change()` נוכל לחשב את שיעור השינוי היחסי בין תקופות —  
למשל לבדוק האם ההספק הכולל גדל או ירד ביחס לחודש הקודם.  
נשתמש בכך כדי לזהות חודשים שבהם ייתכן כשל או שינוי בהתנהגות המערכת.

#### אזורי זמן ו־GroupBy לפי זמן

כאשר המדידות נאספות ממכשירים שונים באזורים שונים,  
נשתמש ב־`tz_localize()` ו־`tz_convert()` כדי לסנכרן בין אזורי זמן.  
ניתן גם להשתמש ב־`groupby(df.index.month)` או `groupby(df.index.day_name())` כדי לחשב ממוצעים תקופתיים.

## תרגילים: סדרות זמן ניסוייות במערכת אנרגיה סולארית

בדוגמה הבאה ננתח סדרת זמן מניסוי פיזיקלי במערכת **תאים סולאריים**.  
נניח שהמערכת מודדת את ההספק היומי הכולל (`net_power`) במשך שלושה חודשים.  
נרצה:
1. להמיר את עמודת הזמן לפורמט DateTimeIndex.  
2. לחשב את **האנרגיה השבועית הכוללת** (Resample).  
3. לנתח את **שינויי הצמיחה החודשיים (MoM)** ולזהות ירידות בייצור האנרגיה.

#### תרגיל 1: עיבוד סדרת זמן וניתוח אנרגיה שבועית במערכת סולארית

בתרגיל זה נבצע סימולציה פשוטה של **מערכת סולארית** המודדת את האנרגיה היומית (kWh) לאורך תקופה של שלושה חודשים.  
נשתמש בכלים של Pandas לעיבוד **סדרות זמן (Time Series)** כדי לחשב את **סך האנרגיה השבועית** שהמערכת הפיקה.

שלבי העבודה:
1. **יצירת נתונים יומיים** — ניצור טווח תאריכים יומי (`pd.date_range`) וערכים סינתטיים של הספק יומי (`net_power_kWh`) המדמים ימי שמש וימי עננות.  
   כל יום עשירי מייצג יום מעונן שבו התפוקה נמוכה בכ־40%.  
2. **המרת עמודת הזמן לפורמט datetime** והגדרתה כ־Index בעזרת `set_index("timestamp")`.  
   פעולה זו מאפשרת לנו להשתמש בפונקציות זמן חכמות של Pandas.  
3. **דגימה מחדש (Resampling)** — נשתמש ב־`resample("W").sum()` כדי לאחד את הנתונים ברזולוציה שבועית ולחשב את **סך האנרגיה השבועית**.  
   כך נוכל לזהות מגמות כלליות, למשל שבועות שבהם ימי שמש רבים הובילו לתפוקה גבוהה.

תרגיל זה מדגים כיצד ניתן לעבור ממדידות יומיות לניתוח רחב יותר — שימושי במיוחד בניסויים מתמשכים או במערכות אנרגיה מבוססות זמן.


In [3]:
# --- Exercise 1: Time-series setup and weekly resampling ---
import pandas as pd
import numpy as np

# Simulated daily solar power output (kWh)
np.random.seed(42)
dates = pd.date_range("2025-01-01", periods=90, freq="D")
# Simulate some cloudy days and clear days
power = np.random.normal(loc=120, scale=20, size=len(dates))
power[::10] *= 0.6  # reduce every 10th day (cloudy)

df = pd.DataFrame({
    "timestamp": dates,
    "net_power_kWh": power
})

# Convert to datetime and set as index
df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df.set_index("timestamp")

# Resample weekly to compute total energy produced per week
weekly_energy = df.resample("W").sum()

print("Weekly total solar energy (kWh):")
display(weekly_energy)


Weekly total solar energy (kWh):


,net_power_kWh
timestamp,
2025-01-05,593.926584
2025-01-12,820.836318
2025-01-19,728.697376
2025-01-26,741.012583
2025-02-02,788.202580
2025-02-09,753.237180
2025-02-16,742.773826
2025-02-23,779.636455
2025-03-02,828.663075


#### תרגיל 2: חישוב שינויי ייצור חודשיים (Month-over-Month Growth)

נחשב את השינוי היחסי בהפקת האנרגיה מחודש לחודש,  
ונזהה חודשים שבהם ייצור החשמל הסולארי ירד — למשל עקב תנאי מזג אוויר או תקלה במערכת.


In [4]:
# --- Exercise 2: Month-over-Month growth analysis ---
# Resample monthly to compute total energy per month
monthly_energy = df.resample("M").sum()

# Compute Month-over-Month (MoM) growth rate in percentage
monthly_energy["MoM_growth_%"] = monthly_energy["net_power_kWh"].pct_change() * 100

# Flag months with negative growth
monthly_energy["production_decline"] = monthly_energy["MoM_growth_%"] < 0

print("Monthly solar energy and growth trends:")
display(monthly_energy)


Monthly solar energy and growth trends:


/tmp/ipykernel_29371/1042207626.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_energy = df.resample("M").sum()


,net_power_kWh,MoM_growth_%,production_decline
timestamp,,,
2025-01-31,3395.899821,NaN,False
2025-02-28,3175.325343,-6.495318,True
2025-03-31,3605.941370,13.561320,False


#### תרגיל 3: התאמה לאזור זמן ובדיקת השפעה על הנתונים

נניח שהמערכת ממוקמת בישראל, אך נתוני הכיול מתקבלים משרת ב־UTC.  
נמיר את אזור הזמן של המדידות (`tz_localize`) ונתאים אותו ל־UTC להשוואה.


In [5]:
# --- Exercise 3: Time zone localization and conversion ---
# Localize timestamps to Asia/Jerusalem
df_local = df.tz_localize("Asia/Jerusalem")

# Convert to UTC
df_utc = df_local.tz_convert("UTC")

print("Localized to Asia/Jerusalem:")
display(df_local.head(2))

print("Converted to UTC:")
display(df_utc.head(2))


Localized to Asia/Jerusalem:


,net_power_kWh
timestamp,
2025-01-01 00:00:00+02:00,77.960570
2025-01-02 00:00:00+02:00,117.234714


Converted to UTC:


,net_power_kWh
timestamp,
2024-12-31 22:00:00+00:00,77.960570
2025-01-01 22:00:00+00:00,117.234714


#### תרגיל 4: GroupBy מודע לזמן — ניתוח ממוצע יומי לפי יום בשבוע

נחשב את ממוצע ההפקה היומית של האנרגיה הסולארית לפי יום בשבוע.  
כך נוכל לבדוק האם קיימת השפעה מחזורית — לדוגמה, האם בימי ראשון נמדדים ערכים נמוכים עקב תחזוקה או הצללה.


In [6]:
# --- Exercise 4: Analyze average daily production by weekday ---
df["weekday"] = df.index.day_name()
weekday_avg = (
    df.groupby("weekday")["net_power_kWh"]
    .mean()
    .reindex(["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"])
)

print("Average daily solar production by weekday (kWh):")
display(weekday_avg)


Average daily solar production by weekday (kWh):


weekday
Sunday       111.324696
Monday       119.241631
Tuesday      117.678589
Wednesday    101.059247
Thursday     116.829803
Friday       108.842895
Saturday     116.934302
Name: net_power_kWh, dtype: float64

`````{admonition} סיכום
:class: tip
בפרק זה למדנו כיצד לעבד ולנתח סדרות זמן של נתונים ניסיוניים:

- להמיר עמודת זמן ל־`DateTimeIndex` ולבצע דגימה מחדש (`resample`) לתקופות שבועיות או חודשיות.  
- לחשב **שיעור שינוי חודשי (MoM)** ולהתריע על ירידות בייצור האנרגיה.  
- לעבוד עם **אזורי זמן שונים** לצורך סנכרון מדידות ממקורות מגוונים.  
- להשתמש ב־`groupby` לפי יום בשבוע כדי לזהות דפוסים מחזוריים.  

שיטות אלו מהוות בסיס חיוני לניתוח נתונים בזמן אמת — כמו מערכות חישה סולאריות, ניסויי פיזיקה רציפים,  
או כל מערכת שבה הזמן הוא גורם מרכזי בהבנת התנהגות התהליך.
`````

In [7]:
import json
from jupyterquiz import display_quiz

quiz_json = \
'''
[{
  "question": "במערכת ניסויית למדידת אנרגיה סולארית נרשמות מדידות יומיות של הספק כולל (net_power_kWh).<br><br>כיצד נכון לחשב את סך האנרגיה השבועית, את קצב השינוי החודשי (MoM), ולזהות חודשים שבהם חלה ירידה בייצור?",
  "type": "many_choice",
  "answers": [
    {
      "answer": "להגדיר את עמודת הזמן כ־DateTimeIndex, לבצע resample('W').sum() לאנרגיה השבועית, ולאחר מכן resample('M').sum().pct_change()×100 לחישוב השינוי החודשי, ולסמן ערכים שליליים כירידה.",
      "correct": true,
      "feedback": "נכון מאוד! זהו תהליך מלא של ניתוח סדרת זמן: אינדוקס לפי זמן, דגימה מחדש שבועית וחודשית, חישוב אחוז שינוי חודשי וזיהוי תקופות ירידה."
    },
    {
      "answer": "להשתמש ב־groupby('month') לחישוב סך האנרגיה, ולאחר מכן לחשב את ההפרש בין החודש הראשון לאחרון בלבד.",
      "correct": false,
      "feedback": "לא מדויק — groupby אינו מטפל ברצף זמנים תקני כמו resample, ולא מאפשר חישוב אוטומטי של שינוי מחודש לחודש."
    },
    {
      "answer": "לחשב את ההפרש בין כל שתי מדידות עוקבות ולחפש ערכים שליליים.",
      "correct": false,
      "feedback": "לא — זה מודד שינוי יומי, לא שינוי חודשי מצטבר. יש להשתמש ב־resample ו־pct_change כדי למדוד שינויי מגמה רחבים."
    },
    {
      "answer": "להשתמש ב־rolling(30).mean() כדי לחשב את האנרגיה החודשית ולזהות ירידות לפי סטיות תקן.",
      "correct": false,
      "feedback": "לא מדויק — rolling מחשבת ממוצעים נעים ולא סכומים חודשיים, ולכן אינה מתאימה לחישוב שינוי חודשי ישיר."
    }
  ]
}]
'''

myquiz = json.loads(quiz_json)
display_quiz(myquiz)


<IPython.core.display.Javascript object>